# Elasticsearch 高度な日本語検索(辞書)
3.2節で説明したElasticsearch サーバーの起動が前提です。  
更に書籍p.103記載の辞書登録手順も済んでいることが前提です。

In [ ]:
# Elasticsearchインスタンスの生成

from elasticsearch import Elasticsearch
es = Elasticsearch()

In [ ]:
# インデックス作成用JSONの定義

create_index = {
    "settings": {
        "analysis": {
            "filter": {
                "synonyms_filter": { # 同義語フィルターの定義
                    "type": "synonym",
                    "synonyms": [ #同義語リストの定義
                            "すし,スシ,鮨,寿司"
                        ]
                }
            },
            "tokenizer": {
                "kuromoji_w_dic": { # カスタム形態素解析の定義
                "type": "kuromoji_tokenizer", # kromoji_tokenizerをベースにする
                    # ユーザー辞書としてmy_jisho.dicを追加  
                    "user_dictionary": "my_jisho.dic" 
                }
            },
            "analyzer": {
                "jpn-search": { # 検索用アナライザーの定義
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer", # 文字単位の正規化
                        "kuromoji_iteration_mark" # 繰り返し文字の正規化
                    ],
                    "tokenizer": "kuromoji_w_dic", # 辞書付きkoromoji形態素解析
                    "filter": [
                        "synonyms_filter", # 同義語展開
                        "kuromoji_baseform", # 活用語の原型化
                        "kuromoji_part_of_speech", # 不要品詞の除去
                        "ja_stop", #不要単語の除去
                        "kuromoji_number", # 数字の正規化
                        "kuromoji_stemmer" #長音の正規化
                    ]
                },
                "jpn-index": { # インデックス生成用アナライザーの定義
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer", # 文字単位の正規化
                        "kuromoji_iteration_mark" # 繰り返し文字の正規化
                    ],
                    "tokenizer": "kuromoji_w_dic", # 辞書付きkoromoji形態素解析
                    "filter": [
                        "kuromoji_baseform", # 活用語の原型化
                        "kuromoji_part_of_speech", # 不要品詞の除去
                        "ja_stop", #不要単語の除去
                        "kuromoji_number", # 数字の正規化
                        "kuromoji_stemmer" #長音の正規化
                    ]
                }
            }
        }
    }
}

# 日本語用インデックス名の定義
jp_index = 'jp_index'

# 同じ名前のインデックスがすでにあれば削除する
if es.indices.exists(index = jp_index):
    es.indices.delete(index = jp_index)

# インデックス jp_doc の生成
es.indices.create(index = jp_index, body = create_index)

In [ ]:
# 分析結果表示用関数

def analyse_jp_text(text):
    body = {"analyzer": "jpn-search", "text": text}
    ret = es.indices.analyze(index = jp_index, body = body)
    tokens = ret['tokens']
    tokens2 = [token['token'] for token in tokens]
    return tokens2

# 関数のテスト
print(analyse_jp_text('関数のテスト'))

In [ ]:
#  リスト3.3.20 辞書登録後の「新幹線はやぶさ」と「はやぶさ」の分析結果

print(analyse_jp_text('新幹線はやぶさ'))
print(analyse_jp_text('はやぶさ'))

In [ ]:
# マッピング定義

mapping =  {
    "properties": {
        "content": {
            "type": "text",
            # インデックス生成時アナライザの設定
            "analyzer": "jpn-index",
            # 検索時アナライザの設定
            "search_analyzer": "jpn-search"
        }
    }
}
es.indices.put_mapping(index = jp_index,   body = mapping)

In [ ]:
# 日本語文書の投入

bodys = [
    { "title": "山田太郎の紹介",
    "name": {
        "last": "山田",
        "first": "太郎"
    },
    "content": "スシが好物です。犬も好きです。"},
    { "title": "田中次郎の紹介",
    "name": {
        "last": "田中",
        "first": "次郎"
    },
    "content": "そばがだいすきです。ねこも大好きです。"},
    { "title": "渡辺三郎の紹介",
    "name": {
        "last": "渡辺",
        "first": "三郎"
    },
    "content": "天ぷらが好きです。新幹線はやぶさのファンです。"}
]

for i, body in enumerate(bodys):
    es.index(index = jp_index, id = i, body = body)

In [ ]:
# リスト 3.3.21 辞書登録後にキーワード「はやぶさ」で検索

# 検索条件の設定
query = {
    "query": {
        "match": {
            "content": 'はやぶさ'
        }
    }
}

# 検索実行
res = es.search(index = jp_index, body = query)

# 結果表示
import json
print(json.dumps(res, indent=2, ensure_ascii=False))